In [1]:
import scrapy
import scrapy.crawler as crawler
from scrapy.utils.log import configure_logging
from multiprocessing import Process, Queue
from twisted.internet import reactor
import logging

# logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger('nlp')
logger.setLevel(logging.INFO)
configure_logging()

In [2]:
# class LinkCheckerSpider(scrapy.Spider):
#     name = 'link_checker'
#     start_urls = ['https://www.state.gov/rss-feeds/']

#     def __init__(self, links_storage):
#         super().__init__()
#         self.links_storage = {'links' : []}
    
#     def parse(self, response):
#         """ Main function that parses downloaded pages """
#         a_selectors = response.xpath("//a")
#         for selector in a_selectors:
#             # Extract the link href
#             link = selector.xpath("@href").extract_first()
#             print('&&&')
#             if 'rss' in link:
#                 links_storage['links'].append(link)
#             # Create a new Request object
# #             request = response.follow(link, callback=self.parse)
# #             # Return it thanks to a generator
# #             yield request

# # the wrapper to make it run more times

    
#     else:
#         logger.info('Links were fetched.')
#         print(links_storage)

In [3]:
# print('first run:')
# run_spider(LinkCheckerSpider)


In [10]:
def fetch_rss_links():
    links_storage = {'links' : []}
    
    class LinkCheckerSpider2(scrapy.Spider):
        name = "link_checker2"
        start_urls = ['https://www.state.gov/rss-feeds/']
        
        def parse(self, response):
            a_selectors = response.xpath("//a")
            for selector in a_selectors:
                # Extract the link href
                link = selector.xpath("@href").extract_first()
                if 'rss' in link:
                    links_storage['links'].append(link)
                    print(link)
    def run_spider(spider):
        def f(q):
            try:
                runner = crawler.CrawlerRunner()
                deferred = runner.crawl(spider)
                deferred.addBoth(lambda _: reactor.stop())
                reactor.run()
                q.put(None)
            except Exception as e:
                q.put(e)

        q = Queue()
        p = Process(target=f, args=(q,))
        p.start()
        result = q.get()
        p.join()
        print(result)
        if result is not None:
            raise result
    run_spider(LinkCheckerSpider2)
    print(links_storage)

fetch_rss_links()

2022-04-26 20:06:13 [scrapy.crawler] INFO: Overridden settings:
{}
2022-04-26 20:06:13 [scrapy.extensions.telnet] INFO: Telnet Password: 90ef181eef89fa05
2022-04-26 20:06:13 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2022-04-26 20:06:13 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrap

https://www.state.gov/rss-feed/africa/feed/
https://www.state.gov/rss-feed/arms-control-and-international-security/feed/
https://www.state.gov/rss-feed/climate-environment-and-conservation/feed/
https://www.state.gov/rss-feed/collected-department-releases/feed/
https://www.state.gov/rss-feed/counterterrorism/feed/
https://www.state.gov/rss-feed/democracy-human-rights-and-labor/feed/
https://www.state.gov/rss-feed/department-press-briefings/feed/
https://www.state.gov/rss-feed/diplomatic-security/feed/
https://www.state.gov/rss-feed/direct-line-to-american-business/feed/
https://www.state.gov/rss-feed/east-asia-and-the-pacific/feed/
https://www.state.gov/rss-feed/economic-energy-agricultural-and-trade-issues/feed/
https://www.state.gov/rss-feed/europe-and-eurasia/feed/
https://www.state.gov/rss-feed/international-expositions/feed/
https://www.state.gov/rss-feed/international-health-issues/feed/
https://www.state.gov/rss-feed/international-organizations/feed/
https://www.state.gov/rss-fe

2022-04-26 20:06:14 [scrapy.core.engine] INFO: Closing spider (finished)
2022-04-26 20:06:14 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 223,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 25311,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'elapsed_time_seconds': 0.528409,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2022, 4, 26, 17, 6, 14, 160182),
 'httpcompression/response_bytes': 140135,
 'httpcompression/response_count': 1,
 'log_count/DEBUG': 1,
 'log_count/INFO': 10,
 'memusage/max': 101806080,
 'memusage/startup': 101806080,
 'response_received_count': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2022, 4, 26, 17, 6, 13, 631773)}
2022-04-26 20:06:14 [scrapy.core.engine] INFO: Spider closed (finished)


None
{'links': []}


In [5]:
# class QuotesSpider(scrapy.Spider):
#     name = "quotes"
#     start_urls = ['https://www.state.gov/rss-feeds/']

#     def parse(self, response):
#         for quote in response.css('div.quote'):
#             print(quote.css('span.text::text').extract_first())

In [6]:
# runner = crawler.CrawlerRunner()
# deferred = runner.crawl(LinkCheckerSpider2)
# deferred.addBoth(lambda _: reactor.stop())
# reactor.run()
# q.put(None)

In [7]:
# intr = runner
# print(intr.settings)
# dir(intr)